# Terminating Conversations Between Agents

We will explore how to terminate a conversation between AutoGen agents.


in any complex, autonomous workflows it’s crucial to know when to stop the workflow. For example, when the task is completed, or perhaps when the process has consumed enough resources and needs to either stop or adopt different strategies, such as user intervention. So AutoGen natively supports several mechanisms to terminate conversations.

Currently there are two broad mechanism to control the termination of conversations between agents:

1. **Specify parameters in ```initiate_chat```**: When initiating a chat, you can define parameters that determine when the conversation should end.

2. **Configure an agent to trigger termination**: When defining individual agents, you can specify parameters that allow agents to terminate of a conversation based on particular (configurable) conditions.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

# Parameters in ```initiate_chat```

This is done by ```max_turns``` parameter .

In [2]:
from autogen import ConversableAgent

cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4o-mini", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4o-mini", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

In [3]:
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.", max_turns=2)

joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

Sure, Joe! Why did the scarecrow win an award? 

Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

That's a classic! Alright, here's one for you: Why don't skeletons fight each other? 

They don't have the guts!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, good one, Joe! Alright, I've got another for you: What do you call fake spaghetti? 

An impasta!

--------------------------------------------------------------------------------


# Agent-triggered termination
Currently, there are two parameters you can configure:

1. ```max_consecutive_auto_reply```: This condition triggers termination if the number of automatic responses to the same sender exceeds a threshold. You can customize this using the ```max_consecutive_auto_reply``` argument of the ```ConversableAgent``` class. To accomplish this the agent maintains a counter of the number of consecutive automatic responses to the same sender. Note that this counter can be reset because of human intervention. We will describe this in more detail in the next chapter.

2. ```is_termination_msg```: This condition can trigger termination if the received message satisfies a particular condition, e.g., it contains the word “TERMINATE”. You can customize this condition using the ```is_terminate_msg``` argument in the constructor of the ```ConversableAgent``` class.

## Using ```max_consecutive_auto_reply```

In the example below lets set ```max_consecutive_auto_reply``` to ```1``` and notice how this ensures that Joe only replies once.


In [4]:
joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4o-mini", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
    max_consecutive_auto_reply=1,  # Limit the number of consecutive auto-replies.
)

result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.")

joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

Sure, Joe! Why did the scarecrow win an award? 

Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

That's a classic! Alright, here's one for you: Why don't skeletons fight each other? 

They don't have the guts!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, good one, Joe! Alright, I've got another for you: What do you call fake spaghetti? 

An impasta!

--------------------------------------------------------------------------------


# Using ```is_termination_msg```

Let’s set the termination message to “GOOD BYE” and see how the conversation terminates.




In [5]:
joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4o-mini", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
    is_termination_msg=lambda msg: "good bye" in msg["content"].lower(),
)

result = joe.initiate_chat(cathy, message="Cathy, tell me a joke and then say the words GOOD BYE.")

joe (to cathy):

Cathy, tell me a joke and then say the words GOOD BYE.

--------------------------------------------------------------------------------
cathy (to joe):

Sure! Here you go: 

Why did the scarecrow win an award? 

Because he was outstanding in his field! 

GOOD BYE!

--------------------------------------------------------------------------------
